In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur

In [3]:
augurApp = augur.Application('../augur.config.json')
list1,path = augurApp.github_issues()
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)

In [ ]:
if "notebooks" in os.getcwd():
    os.chdir("..")
path = os.getcwd() + '/runtime/git_repos.csv'
print(path)
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("owner,repo_url\n")
    #file.write("nodejs,\"CTC\"\n")
    #file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

In [ ]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
token = 'e3f0dd3ccf0174ba4fe351c7c12a275b666f2c34'
own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'

# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning
repo = GitHub(owner=own,repository=repo_url,api_token=token)

#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
k=0
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df = pd.DataFrame(columns=columns1)
item = 1
df.to_sql(name="github_issues", con=db,\
          if_exists='replace',index=False,
           dtype={'augurmsgID': s.types.Integer,#0
                  'backend_name': s.types.VARCHAR(length=300),#1
                  'repo_link': s.types.VARCHAR(length=300),#2
                  'owner': s.types.VARCHAR(length=300),#3
                  'repo': s.types.VARCHAR(length=300),#4
                  'subject': s.types.VARCHAR(length=300),#5
                  'status': s.types.VARCHAR(length=10),#6
                  'category': s.types.VARCHAR(length=10),#7
                  'issue_number': s.types.Integer,#8
                  'timestamp': s.types.Integer,#9
                  'issue_id': s.types.Integer,#10
                  'user': s.types.VARCHAR(length=100),#11
                  'body':s.types.TEXT#12              
           })
for issues in repo.fetch():
    # Create the object (dictionary) to upload to ElasticSearch
    # Create the object (dictionary) to upload to ElasticSearch
    #if(i==90):
    if 'pull_request' in issues['data']:
        #print(issues['data']['number'])
        continue
    else:
        #if(issues['data']['state'] == 'closed' and issues['data']\
        #   ['number'] == 165): 
        text = issues['data']['body']
        user = issues['data']['user']['login']
        num = issues['data']['number']
        id_num = issues['data']['id']
        #print(id_num)
        store = [item,issues['backend_name'],issues['tag'],own,repo_url,\
                 issues['data']['title'],issues['data']['state'],\
                 issues['category'],num,issues['timestamp'],\
                 id_num,user,text]
        #print(store[:])
        #print(store[6])
        #print("Issue number: ",issues['data']['number'])
        #if(k > 1):
            #k+=1
            #continue
        '''print("Backend_name: ", issues['backend_name'])
        print("Category: ", issues['category'])
        print("Subject: ", issues['data']['title'])
        print("State: ", issues['data']['state'])
        print("First body:",issues['data']['body'])
        print("User: ", issues['data']['user']['login'])
        print("\n\n\n")'''
        df_user = pd.DataFrame([store],columns=columns1)
        df = df.append(df_user)
        item+=1
        for i in range(len(issues['data']['comments_data'])):
            temp=0
            store[0] = item
            store[8] = issues['data']['number']
            store[10] = issues['data']['id']
            store[11] = issues['data']['comments_data'][i]\
            ['user_data']['login']
            store[12] = issues['data']['comments_data'][i]['body']
            df_user = pd.DataFrame([store],columns=columns1)
            df = df.append(df_user)
            #print(issues['data']['comments_data'][i]\
            #      ['user_data']['login'])
            #print(issues['data']['comments_data'][i]['body'])
            item+=1
          
    if(df.shape[0] > 1000):
        df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
        df = pd.DataFrame(columns=columns1)
        print("Broken")
        #break
    #print(issues['data'][''])
    if(k%50 == 0):
        print(k)
    k+=1
    #if(k == 2):
    #    break
#print(df)
if(df.shape[0] < 1000):
    df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
    df = pd.DataFrame(columns=columns1)
    print("Broken")
print("Total Number of issues: ", k)
print(df.shape[0])